In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [2]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        if d_type=='train':
            with open('../../data/chat_feature_train.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='val':
            with open('../../data/chat_feature_val.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='test':
            with open('../../data/chat_feature_test.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        
        with open('../../data/video_statistic_features_one2.pickle',"rb") as f2:  
            self.image_result=pickle.load(f2)
        with open('../../data/audio_entropy_emd.pickle',"rb") as f3:  
            self.audio_result=pickle.load(f3)
        with open('../../label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.chat_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.chat_result[game_id]):
#                     s_window+=((self.chat_result[game_id][vframe+idx]))#vframe의 chat
                    s_window+=list(self.audio_result[game_id][(vframe+idx)*10:(vframe+idx+1)*10])#vframe의 audio
                    s_window+=[(self.image_result[game_id][vframe+idx])]#vframe의 image
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*11
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [5]:
train=Mul_data('train')
val=Mul_data('val')
print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[ 5.28188175e-04, -2.60772944e-03, -6.42303089e-03,
         5.93419674e-03,  4.28576542e-03,  3.60430944e-03,
         1.76061821e-03, -3.37451883e-03,  3.59619475e-03,
        -4.46354486e-03,  1.37379766e-03],
       [-7.07925046e-04,  7.65639497e-03,  6.60309501e-03,
        -1.16132757e-03, -1.27804309e-02,  1.37522797e-02,
         8.39392149e-03, -6.83272836e-03, -1.36067919e-02,
        -1.21897105e-02,  1.37379766e-03],
       [-4.37973586e-03,  4.64145782e-03,  9.35147676e-03,
         5.34418137e-03, -7.14635322e-03,  6.27936901e-03,
        -3.65286729e-03, -5.03458177e-03, -1.22952435e-03,
         4.21928000e-03,  2.00727582e-03],
       [-3.91807628e-03,  1.14446836e-04,  2.56711422e-03,
         3.25942224e-03,  3.36269885e-03,  2.55329375e-03,
        -7.16273585e-04, -2.33919975e-03, -3.58056417e-03,
        -4.01098511e-03,  7.20083714e-04],
       [-3.39250482e-03, -2.00500236e-03, -2.74149209e-04,
         4

In [3]:
input_size=11
hidden_size=23
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        feature,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        out = self._lin(feature[:,-1,:])
        return feature[:,-1,:]

In [4]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [5]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
weight_dir='./2features_winlossone2+ent_emd_undersampling/'
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(200):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.7171695232391357,p 0, r 0, f 0

1
epoch 1 train_loss : 0 , val_loss : 0.6672632098197937,p 0, r 0, f 0

2
epoch 2 train_loss : 0 , val_loss : 0.3891659080982208,p 0.5873655913978495, r 0.6752759381898454, f 0.6282604230848223

3
epoch 3 train_loss : 0 , val_loss : 0.4389633536338806,p 0.5298804780876494, r 0.704635761589404, f 0.6048891415577031

4
epoch 4 train_loss : 0 , val_loss : 0.47662296891212463,p 0.5219376936878161, r 0.7064017660044151, f 0.6003189194259452

5
epoch 5 train_loss : 0 , val_loss : 0.4261067509651184,p 0.5566205182818601, r 0.6922737306843267, f 0.6170798898071626

6
epoch 6 train_loss : 0 , val_loss : 0.4146990478038788,p 0.5486910994764398, r 0.6940397350993377, f 0.6128654970760234

7
epoch 7 train_loss : 0 , val_loss : 0.4289664924144745,p 0.5339039946290701, r 0.7022075055187638, f 0.606598016781083

8
epoch 8 train_loss : 0 , val_loss : 0.3833683729171753,p 0.5946999220576773, r 0.6737306843267108, f 0.63175326019457

epoch 67 train_loss : 0 , val_loss : 0.4082333743572235,p 0.5731729215298976, r 0.704635761589404, f 0.6321417962174473

68
epoch 68 train_loss : 0 , val_loss : 0.4222363531589508,p 0.5599512025095852, r 0.7092715231788079, f 0.6258278145695364

69
epoch 69 train_loss : 0 , val_loss : 0.40574729442596436,p 0.5658545197740112, r 0.7075055187637969, f 0.6288012556405729

70
epoch 70 train_loss : 0 , val_loss : 0.41614577174186707,p 0.5487057220708447, r 0.7112582781456953, f 0.6194962507210152

71
epoch 71 train_loss : 0 , val_loss : 0.42086029052734375,p 0.5515710382513661, r 0.7130242825607064, f 0.6219911419218179

72
epoch 72 train_loss : 0 , val_loss : 0.4442111849784851,p 0.529985372988786, r 0.7198675496688741, f 0.6105026677899467

73
epoch 73 train_loss : 0 , val_loss : 0.4304145276546478,p 0.5542896599344036, r 0.7088300220750552, f 0.6221059769446866

74
epoch 74 train_loss : 0 , val_loss : 0.40246862173080444,p 0.5507839127471029, r 0.7134657836644591, f 0.6216580111559915

7

epoch 133 train_loss : 0 , val_loss : 0.4025893211364746,p 0.5777737816939399, r 0.7092715231788079, f 0.6368050738281636

134
epoch 134 train_loss : 0 , val_loss : 0.4127669930458069,p 0.5577219086711134, r 0.7198675496688741, f 0.6285053483665799

135


In [11]:
weight_dir='./2features_winlossone2+ent_emd_undersampling/'

test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 4 23 0 tensor(9) tensor(5)
9 0 23 0 tensor(9) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 0 23 0 tensor(9) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
2 0 16 14 tensor(2) tensor(16)
16 0 15 1 tensor(16) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 7 13 1 tensor(18) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 14 18 0 tensor(14) tensor(0)
0 5 27 0 tensor(5) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 25 7 tensor(0) tensor(7)
0 0 14 18 tensor(0) tensor(18)
0 5 19 8 tensor(5) tensor(8)
0 21 11 0 tensor(21) tensor(0)
13 1 18 0 tenso

0 20 12 0 tensor(20) tensor(0)
14 9 9 0 tensor(23) tensor(14)
5 0 21 6 tensor(5) tensor(11)
7 2 20 3 tensor(9) tensor(10)
0 0 30 2 tensor(0) tensor(2)
0 4 28 0 tensor(4) tensor(0)
0 14 18 0 tensor(14) tensor(0)
0 9 23 0 tensor(9) tensor(0)
15 17 0 0 tensor(32) tensor(15)
8 0 23 1 tensor(8) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 7 25 0 tensor(7) tensor(0)
0 11 21 0 tensor(11) tensor(0)
3 10 19 0 tensor(13) tensor(3)
22 0 0 10 tensor(22) tensor(32)
18 0 12 2 tensor(18) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 18 14 0 tensor(18) tensor(0)
29 3 0 0 tensor(32) tensor(29)
11 5 15 1 tensor(16) tensor(12)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 18 14 0 tensor(18) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
18 1 10 3 tensor(19) tensor(21)
8 1 5 18 tensor(9) tensor(26)
0 0 32 0 tensor(0) tensor(0)
0 0 30 2 tensor(0) tensor(2)
18 0 12 2 tensor(18) tensor(20)
0 0 

13 0 9 10 tensor(13) tensor(23)
0 7 16 9 tensor(7) tensor(9)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 25 7 tensor(0) tensor(7)
0 0 26 6 tensor(0) tensor(6)
0 0 20 12 tensor(0) tensor(12)
0 0 12 20 tensor(0) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 0 23 0 tensor(9) tensor(9)
7 25 0 0 tensor(32) tensor(7)
2 3 27 0 tensor(5) tensor(2)
22 10 0 0 tensor(32) tensor(22)
26 6 0 0 tensor(32) tensor(26)
8 0 24 0 tensor(8) tensor(8)
19 4 5 4 tensor(23) tensor(23)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
18 0 13 1 tensor(18) tensor(19)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 2 23 0 tensor(9) tensor(7)
12 5 15 0 tensor(17) tensor(12)
1 0 30 1 tensor(1) tensor(2)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) ten

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 10 22 tensor(0) tensor(22)
19 0 3 10 tensor(19) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 1 24 0 tensor(8) tensor(7)
1 0 30 1 tensor(1) tensor(2)
0 0 32 0 tensor(0) tensor(0)
16 2 13 1 tensor(18) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 18 14 0 tensor(18) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 3 22 1 tensor(9) tensor(7)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
2 2 

In [9]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [12]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('../../label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6015228426395939, recall : 0.6220472440944882, f1 : 0.6116129032258063, accuracy : 0.8778409090909091
102844341902586509
precision : 0.54739336492891, recall : 0.9130434782608695, f1 : 0.6844444444444445, accuracy : 0.9003741814780168
102844401152267937
precision : 0.608424336973479, recall : 0.9701492537313433, f1 : 0.74784276126558, accuracy : 0.8851026649191787
102844212430927059
precision : 0.6129032258064516, recall : 0.6621212121212121, f1 : 0.6365622723962128, accuracy : 0.8687877991059689
102844412708953395
precision : 0.625, recall : 0.8695652173913043, f1 : 0.7272727272727273, accuracy : 0.9124513618677043
102844212429944013
precision : 0.642023346303502, recall : 0.889487870619946, f1 : 0.7457627118644068, accuracy : 0.8900293255131965
102844341912679064
precision : 0.5105105105105106, recall : 0.7522123893805309, f1 : 0.6082289803220036, accuracy : 0.8871134020618556
102844235753749959
precision : 0.5934718100890207, recall : 0.5089058524173

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571

In [10]:
weight_dir='./2features_winlossone2+ent_emd_undersampling/'
test=Mul_data('train')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output=model(inputs)
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                print(g)
                result[g]=[output[idx].tolist()]
            else:
                result[g].append(output[idx].tolist())

data load fin
102844412722519367
102844212429550795
102844401151219358
102844401154430631
102844412717014335
102844401153971877
102844224148503678
102844412722847048
102844401152857762
102844412707380528
102844212431516886
102844283027925085
102844412716227901
102844412710001974
102844294670878922
102844294670551241
102844283023599703
102844412704496937
102844235751783874
102844401152071328
102844412709674293
102844401153447587
102844224148896895
102844235746868664
102979081290790284
102844283027531868
102844212431975640
102844401155937960
102844212429092040
102844341906649746
102844412706987311
102844412721339716
102844212430402768
102844341905011343
102844235753356742
102844235750997440
102844412709346612
102844412705217835
102844235752963525
102844412712164667
102844412705545516
102844341912220311
102844341907370644
102844235749424575
102844212429419722
102844294669568199
102844212431779031
102844294666422466
102844224146472059
102844212428895431
102844212429747404
10284423574870367

In [11]:
with open('./2features_winlossone2+ent_emd_undersampling/lstm_feature_train2.json','a') as f:
    json.dump(result,f)